## Extract one OC with the DBSCAN parameters

In [15]:
using PyCall
using Statistics , StatsBase
using  DataFrames, Printf , CSV , Query

import PyPlot
sns= pyimport("seaborn")

## directory
rootdir = ENV["GAIA_ROOT"]
wdir    = "$rootdir/products"
votdir  = "$rootdir/products/votable.2020"
plotdir = "$rootdir/products/test"
sclist  = "$rootdir/e2e_products/sc-list-2020.csv"

push!(LOAD_PATH,"$rootdir/run/src")
using GaiaClustering

apy= pyimport("astropy")
coord= pyimport("astropy.coordinates")

cd(wdir)

In [16]:
function _get_OC(df , dfcart, labels , labelmax)
    ra= df.raw[1, labels[labelmax]]
    dec= df.raw[2,labels[labelmax]]
    l= df.data[1, labels[labelmax]]
    b= df.data[2,labels[labelmax]]
    d= df.data[3,labels[labelmax]]
    pmra= df.raw[6, labels[labelmax]]
    pmdec= df.raw[7, labels[labelmax]]
    X= dfcart.data[1, labels[labelmax]]
    Y= dfcart.data[2, labels[labelmax]]
    Z= dfcart.data[3, labels[labelmax]]
    vl= df.data[4,labels[labelmax]]
    vb= df.data[5,labels[labelmax]]
    vrad= df.raw[13,labels[labelmax]]
    gbar= df.raw[10,labels[labelmax]]
    rp= df.raw[11,labels[labelmax]]
    bp= df.raw[12,labels[labelmax]]
    ag= df.raw[14,labels[labelmax]]

    oc= DataFrame(ra=ra,dec=dec,l=l,b=b, distance=d,pmra=pmra, pmdec=pmdec, X=X,Y=Y,Z=Z,vl=vl,vb=vb,vrad=vrad,gbar=gbar,rp=rp,bp=bp, ag=ag)

    return(oc)
end

function _getdata(voname, wghtblck)
    data       = read_votable(voname)
    df         = filter_data(data, [0.,4000])
    dfcart     = add_cartesian(df)
    blck       = [[1,2,3],[4,5], [6,7,8]]
    norm       = "identity"

    dfcartnorm , scale8 = normalization_PerBlock(dfcart, blck, wghtblck , norm, false)
    return(df, dfcart , dfcartnorm)
end

_getdata (generic function with 1 method)

In [17]:
## return the "best cluster"

function _find_cluster_label(labels, df::GaiaClustering.Df, dfcart::GaiaClustering.Df ,
    aperture2d = 1.5, maxaperture2d = 15, aperturev = 3.0, maxaperturev = 20, nboot = 30)
    
    ### metrics of the clusters
    q2d = metric2(dfcart, labels, "spatial2d" , aperture2d, maxaperture2d, nboot)
    q3d = metric2(dfcart, labels, "spatial3d" , aperture2d, maxaperture2d, nboot)     #### Added
    qv  = metric2(dfcart, labels, "velocity" , aperturev, maxaperturev, nboot)
    qp, qa = metric2(dfcart, labels, "HRD" )

    nlab = []
    for ilab in labels
        push!(nlab,length(ilab))
    end
    
    #### metric for the number of stars in the cluster
    qn = []
    for nl in nlab
        push!(qn,log10(nl))
    end
    
    qc= []
    for i in 1:length(nlab)
        k1 = q2d[i][1]
        k1bis = q3d[i][1]
        k2 = qv[i][1]
        k3 = qa[i][1]
        k4 = qn[i]
        ############### Composite metric ###
        qq = (2k1 + k1bis + 3k2 + k3 + k4) / 8.0
        # qq = (3k1 + k1bis + 3k2 + k4) / 8.0
        ###############
        push!(qc,qq)
    end
    
    println(qc)
    bestlabel= findmax(qc)[2]
    println("best label $bestlabel")
    
    return(bestlabel, nlab[bestlabel])
end

_find_cluster_label (generic function with 6 methods)

In [18]:
## function to compute distance of detected cluster to "standard" position
##

function _get_pos(votname, sclist)
    u= apy.units
    scdata= CSV.read(sclist,delim="|")
    
    nf= findlast("deg", votname)[1]-5  ## deg-5 chars
    name= votname[1:nf]
    println(name)
    
    q = @from i in scdata begin
     @where i.name == name
     @select {ra=i.ra, dec= i.dec}
     @collect DataFrame
    end
    
    if length(q.ra) > 0
        println(q)
        ra= q.ra[1] ; dec= q.dec[1]
        co= "$ra $dec"
        c= coord.SkyCoord(co, unit=(u.hourangle, u.deg))
    
        α= c.ra
        δ= c.dec 
    else
        α= [0.]
        δ= [0.]
        println("## source coordinates not found ...")
    end
    
    return(α[1],δ[1])
end

## angular distance to the cluster litterature center...
function _get_angle2pos(oc,α_c, δ_c )
    α= median(oc.ra)
    δ= median(oc.dec)
        
    angdist= angle4sphere(α, δ ,α_c, δ_c )
    return(angdist)
end

## analyze some properties of the cartesian coordinates for the extraction
function _votinfo(dfcart)
    cart= DataFrame(X=dfcart.data[1,:], Y=dfcart.data[2,:], Z=dfcart.data[3,:])
    
    q = @from i in cart begin
     @where i.X < 1010 &&  i.X > 990
     @select {x= i.X, y= i.Y, z=i.Z}
     @collect DataFrame
    end
    
    coefxym= minimum(cart.Y ./ cart.X)
    coefxzm= minimum(cart.Z ./ cart.X)
    coefxyM= maximum(cart.Y ./ cart.X)
    coefxzM= maximum(cart.Z ./ cart.X)
    
    println("### Max coef. Y/X: $coefxym $coefxyM")
    println("### Max coef. Z/X: $coefxzm $coefxzM")

    PyPlot.plt.figure(figsize=(13.0,12.0))

    PyPlot.plt.subplot(2, 1, 1 , ylim=[100,450], xlim=[-50,50])
    PyPlot.plt.scatter(cart.Y, cart.X, s = 0.1 )
    PyPlot.plt.xlabel("Y (pc)")
    PyPlot.plt.ylabel("X (pc)")
    PyPlot.plt.grid(true)
    
    PyPlot.plt.show()

end

_votinfo (generic function with 1 method)

In [19]:
votname= "NGC 869-3.0deg.vot"

w3d=  4.4
wvel=   8.3
whrd=  2.9

wght= [w3d, wvel, whrd]
df , dfcart , dfcartnorm = _getdata(votdir*"/"*votname, wght)

## info
println("## vot info...")
_votinfo(dfcart)

eps     =  3.1
min_cl  = 19
min_nei = 8


labels = clusters(dfcartnorm.data ,eps  , 20, min_nei, min_cl)
labelbest, nstars= _find_cluster_label(labels, df, dfcart)
# labelmax , nmax = find_cluster_label(labels)

α_c , δ_c = _get_pos(votname, sclist)

oc= [] ; ang= []
for i in 1:length(labels)
    oci= _get_OC(df , dfcart, labels , i)
    angle_c= _get_angle2pos(oci, α_c, δ_c)
    push!(oc,oci)
    push!(ang,angle_c)
end

## Votable /home/stephane/Science/GAIA/products/votable.2020/NGC 869-3.0deg.vot read
## Filtering done ...
## Stars selected: 135000
## Cartesian transformation done ...
## Normalization identity done...
### [1pc,1pc,1pc,1km/s,1km/s,1mag,1mag,1mag] equivalent to [0.2959157811470433, 0.2959157811470433, 0.2959157811470433, 0.5582047689819226, 0.5582047689819226, 0.19503540121055127, 0.19503540121055127, 0.19503540121055127]
##
## vot info...
### Max coef. Y/X: -0.05101973798442552 0.05396149601193421
### Max coef. Z/X: -0.05122308170671857 0.0535326336524231
Any[2.196646570722521, 1.0511644897550592, 0.6649513868933387, 0.6904605636542018, 0.9147156672644217, 0.72336840762075, 1.9810817383605697, 1.0571018583616585, 0.6618553060346124, 2.9000017565246474, 1.2538034292168883, 1.5710103672178002, 1.5564054729604473, 1.1547492895093452, 0.8252222143198258, 0.6582313515322857, 0.9614664022625252, 1.3139781012624923, 0.9564233090785385, 1.3469361100041832, 0.800316094223498, 0.78180679537418

LoadError: ArgumentError: provide a valid sink argument, like `using DataFrames; CSV.read(source, DataFrame)`

In [20]:
## plot_cluster

function _plot_oc(df, angdist= 0)
    PyPlot.plt.figure(figsize=(13.0,12.0))

    PyPlot.plt.subplot(3, 3, 1 , xlim = [-20,20] , ylim = [-20,20])
    PyPlot.plt.scatter(df.Y .- mean(df.Y), df.Z .- mean(df.Z) , s = 1.0 )
    PyPlot.plt.xlabel("Y (pc)")
    PyPlot.plt.ylabel("Z (pc)")
    PyPlot.plt.grid(true)
    
    PyPlot.plt.subplot(3, 3, 2 , ylim = [-20,20])
    PyPlot.plt.scatter(df.X, df.Z .- mean(df.Z)  , s = 1.0 )
    PyPlot.plt.xlabel("X (pc)")
    PyPlot.plt.ylabel("Z (pc)")
    PyPlot.plt.grid(true)
    
    PyPlot.plt.subplot(3, 3, 4 , xlim = [-20,20])
    PyPlot.plt.scatter(df.Y .- mean(df.Y) , df.X, s = 1.0 )
    PyPlot.plt.xlabel("Y (pc)")
    PyPlot.plt.ylabel("X (pc)")
    PyPlot.plt.grid(true)
    
    PyPlot.plt.subplot(3, 3, 3 )
    PyPlot.plt.scatter(df.X, df.vrad , s = 1.0 )
    PyPlot.plt.xlabel("X(pc)")
    PyPlot.plt.ylabel("Vrad (km/s)")
    PyPlot.plt.grid(true)    
        
    PyPlot.plt.subplot(3, 3, 5)
    PyPlot.plt.axis("off")
    ## text to display
    text =[]
    txt = @sprintf("N stars: %3d",length(df.X)) ; push!(text,txt)
    txt = @sprintf("Distance: %3.1f (pc) +/- %3.1f",median(df.distance), std(df.distance)) ; push!(text,txt)
    txt = @sprintf("Angular distance: %3.2f (degree)", angdist) ; push!(text,txt)
    txt = @sprintf("l: %3.3f (degree)",median(df.l)) ; push!(text,txt)
    txt = @sprintf("b: %3.3f (degree)",median(df.b)) ; push!(text,txt)  
    txt = @sprintf("Vl: %3.2f (km/s)",median(df.vl)) ; push!(text,txt) 
    txt = @sprintf("Vb: %3.2f (km/s)",median(df.vb)) ; push!(text,txt)
    txt = @sprintf("Vr: %3.2f (km/s, no masking)",median(df.vrad)) ; push!(text,txt)
    txt = @sprintf("X disp.: %3.2f (pc)", std(df.X)) ; push!(text,txt)
    txt = @sprintf("Y disp.: %3.2f (pc)", std(df.Y)) ; push!(text,txt)  
    txt = @sprintf("Z disp.: %3.2f (pc)", std(df.Z)) ; push!(text,txt)
    txt = @sprintf("Vl disp.: %3.2f (km/s) ", std(df.vl)) ; push!(text,txt)
    txt = @sprintf("Vb disp.: %3.2f (km/s)", std(df.vb)) ; push!(text,txt)
    txt = @sprintf("Vr disp.: %3.2f (km/s)", std(df.vrad)) ; push!(text,txt)
    show_text(-0.01,0.0, text , 1.0)
    
    PyPlot.plt.subplot(3, 3, 7 )
    PyPlot.plt.axis("on")    
    PyPlot.plt.scatter(df.gbar-df.rp, -df.gbar , s = 1.0 )
    PyPlot.plt.xlabel("G-Rp")
    PyPlot.plt.ylabel("G")
    PyPlot.plt.grid(true)
    
    PyPlot.plt.subplot(3, 3, 8 )
    PyPlot.plt.scatter(df.vl, df.vb , s = 1.0 )
    PyPlot.plt.xlabel("Vl (km/s)")
    PyPlot.plt.ylabel("Vb (km/s)")
    PyPlot.plt.grid(true)
    
end

_plot_oc (generic function with 2 methods)

In [21]:
for i in 1:length(labels)
    _plot_oc(oc[i], ang[i])
    PyPlot.savefig(plotdir*"/test_label_$i.png")
end

LoadError: UndefVarError: oc not defined